In [ ]:
import pandas as pd
import numpy as np
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import porter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

import itertools


import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')

## Loading from MongoDB

In [ ]:
#Launches Mongo Client
from pymongo import MongoClient

client = MongoClient()
biased_news = client.events.biased_news

In [ ]:
#Create news bias events
db = client.events
biased_news = db.biased_news

In [ ]:
#check duplicate articles
list(biased_news.aggregate([{'$group' : {'_id': '$title', 'count': {'$sum': 1}}},
    {'$match': {'count': {'$gte': 2}}},
    ]))

In [ ]:
#Create cursor
cursor = biased_news.find()

In [ ]:
#Loads from Mongo
true_df = pd.DataFrame(list(cursor))

In [ ]:
#Delete columns and drop null values
del true_df['sub_title']
del true_df['_id']
true_df = true_df.dropna(how='any')

In [ ]:
#Iterates through dataframe and drops redundant articles
for title in eliminate.index:
    huh = true_df['title'] == title
    wha = true_df[huh]
    label = wha.index
    true_df = true_df.drop(labels=label[1:],axis=0)

## Creating a corpus

In [ ]:
# Creates a list of stop words
stopwords = stopwords.words()

In [ ]:
def clean_text(text):
    '''Removes stop words and changes word to stem words'''
    cleaned_text = []
    for post in text:
        cleaned_words = []
        for word in post.split():
            low_word = word.lower()#stemmer.stem(word.lower())
            if low_word not in stopwords:
                cleaned_words.append(low_word)
        cleaned_text.append(' '.join(cleaned_words))
    return cleaned_text

In [ ]:
#Clean the text
cleaned_text = clean_text(true_df.body)

In [ ]:
def drop_nouns(text_list):
    #Drops the nouns
    no_nouns = []
    cleaned_text = []
    phrases = []
    for x in text_list:
        words = pos_tag(word_tokenize(x))
        werdz3 = ['NNP', 'NN', 'NNP', 'NNPs', 'NNS', ',', '.', ':', '(', ')', '#', '``']
        werdz = [s for s in words if s[-1] not in werdz3]
        say_no = ['@','[', ']', 'amp', 'window', 'open','click', 'googletag', 'gpt', 'linkitem', 'googletag', 'getelementbyid',
        'config', 'ldadinit', 'advertis', 'typeof', 'adsdiv', 'fjs', 'js', 'http', 'com', 'awr', 'new', 'function', 'div',
        'ad', 'script', 'typeof', 'nr_is_logged_in', 'undefined', 'adsdiv', 'sharebox_260x60', 'ifr', 'jwplayer', 'jwp',
        'pubdate', 'adunit', 'adwidth', 'www', 'bit.ly/2jpexyr', 'googletag.cmd.push', 'googletag.display', '\'div-gpt-ad-1415299254516-0 \'',
        'open', 'opening', 'opens', 'opened', 'alabama', 'loading', 'email', 'advertise', 'apps', 'closed', 'help', 'publish',
        'rendered', 'undefined', 'adsdiv', 'reloadcount', 'adsdiv.reloadcount', 'window.orignetid', '\'undefined', 'window.origadsplid', 'needsrecovery',
        'io_c3sd.ads', 'elem', 'box', 'box.offsetheight', 'box.style.marginright', 'marginright', 'ad_sharebox_260x60', 'trump',
        'russia', 'fox', 'alerts', 'facebook', 'reuters', 'rsize', 'i+= resize /scr+ipt', 'ipt', 'scr', 'script', 'window.adsetsynccalled',
        'adsetsynccalled', 'recoveryid', 'enablequeue', 'slotrenderended', 'beast', 'daily beast', 'clicking', 'subscribed', 'korea', 'north korea', 'south korea',
        'breitbart', 'epa', 'labelmapping', 'comey', 'scotus', 'republicans', 'cohen', 'iran', 'syria', 'eagles', 'april', 'colorado', 'fbi',
        'haspel', 'wedding', 'puerto', 'rico', 'puerto rico', 'huffpost', 'donald', 'don', 'nra', 'sachs', 'kelly', 'facebook', 'mohammed',
        'div-gpt-ad-1415299254516-0 ', '\'div-gpt-ad-1415299254516-0 \'', '\'slotrenderended \'', 'slotrenderended', 
        '\'.single-post # div-gpt-ad-1415299254516-0 \'', 'slotrenderended', 'ldadinit', 'roseanne', 'israel', 'gaza', 
        'china', 'beijing', 'samantha', 'bee', 'valierie', 'subscribe', 'prelimmonth', 'ivanka', 'documentcloud', 'melania', 
        '(', ')', '-', ',', '.', '!', '"', '\'', 'var', "\'div-gpt-ad-1415299254516-0", 'div-gpt-ad-1415299254516-0', '\'.single-post'
        '\'slotrenderended', 'funct', '.contents', 'googletag.pubads', "\'slotrenderended", "\'.single-post", "'div-gpt-ad-1403197269028-0"
        , 'line-height', '>', 'adtech-adspot', '//', '/style', 'overrid', 'window.adsetplid', 'adid', '||', 'adtech_call_typ',
        'by_request', 'adtech_call_typ', 'iframe_proxy', 'ifr.offsetwidth', "ifr.offsetheight", "'jquery", "'readytorecover",
        'args', "waitforglobal", 'arg', "elem.contains", "'reloadad", 'els', "'adtech", 'refreshr', "'none", 'collapsed', 
        'fc', 'f', 'ajax', 'needsrecoveri', 'i+=', '&', '<', '=', 'http', 'https', "'http", "'http", "'script", 'twitter', 
        '/.test', "'https", 'newsletter-inline-widget', 'margin-bottom', '15px', 'font-weight', 'font-size', '12pt', 'div.mc-field-group',
        'padding-bottom', 'padding-right', 'input.mc-input', '.newsletter-inline-widget', 'font-family', '.wpcf7-form-control.wpcf7-text',
        '7e7e7e', 'arial', '.wpcf7', '.wpcf7-form-control.wpcf7-text', '.newsletter.widget__contain', '.wpcf7-form-control.wpcf7-submit',
        'box-shadow', 'text-shadow', 'letter-spacing', '.newsletter.widget__wrap', '.newsletter.widget__head', 'text-transform', 
        'max-width', 'td.first', 'padding-left', 'border-radius', 'hr.divid', 'p.subtext','==typeof', '.gettime', 'window.outerwidth',
        'rcel.async', '/**', 'url', "'data-timestamp", '+new', 'd.head', 'javascript', 'php_widget-140', 'php_widget-104', '.today-on-the-show-cont',
        '.alignleft', 'margin-right', 'margin-top', '.alignright', "'/wp-content/uploads/static/tots.html", "'.today-on-the-show-content",
        "'.today-on-the-show-cont", '.html', 'img.hero-ad-speci', 'fa', 'fa-chevron-down', 'usercollapsetext', 'newsletter-side-widget', 
        'ul.stansberry-form', 'text-align', 'margin-left:0', 'margin-bottom:5px', 'border-top', '-webkit-border-radius', '-khtml-border-radius',
        'border-width', 'border-style', 'border-right', 'padding-left:8px', '.textsiz', 'margin-top:5px', 'ul.links_list', 'img.breaking_imag', 
        'adtech_sharebox_260x60', 'advertisement', '===', 'kraken__adblock.active', 'i=0', 'id=', 'style=', '//www.documentcloud.org/documents/4434037-hhrg-115-if00-wstate-zuckerbergm-20180411.js', 
        '//assets.documentcloud.org/documents/4434037/hhrg-115-if00-wstate-zuckerbergm-20180411.pdf', 'br', 'lt', '//assets.documentcloud.org/documents/4434037/hhrg-115-if00-wstate-zuckerbergm-20180411.txt',
        '//www.documentcloud.org/documents/4425618-van-der-zwaan.js', '//assets.documentcloud.org/documents/4425618/van-der-zwaan.pdf', '//assets.documentcloud.org/documents/4425618/van-der-zwaan.txt',
        'subscrib', 'window.datawrapper', '.embeddeltas', 'datawrapper-height', '=typeof', '==', 'div-gpt-ad-inline_1_mobile', '.jw-player-contain', "'jwp",
        '.playlist', 'episode.title', 'ns_st_st=\\', 'ns_st_pu=\\', 'ns_st_pr=\\', 'ns_st_ep=\\', 'ns_st_ia=\\', 'div-gpt-ad-native_mobile', 'div-gpt-ad-inline_2_mobile',
        'div-gpt-ad-inline_3_mobile', 'ns_st_sn=\\', 'ns_st_en=\\', 'ns_st_ge=\\', 'ns_st_ce=\\', 'ns_st_ddt=\\', 'ns_st_tdt=\\', 'div-gpt-ad-native', 'div-gpt-ad-inline_4_mobile',
        '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/slideshow\\/596990', '\\u0000*\\u0000links', '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/media', '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/categories',
        '\\/\\/api.w.org\\/term', '\\/\\/www.nationalreview.com\\/wp-json\\/wp\\/v2\\/tags', '\\/\\/www.nationalreview.com\\/photos\\/prince-harry-meghan-markle-royal-wedding\\/', '2fwww.nationalreview.com', 'text=harri', '2fprince-harry-meghan-markle-royal-wedding',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-26.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-8.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-37.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-110.jpg',
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-111.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-45.jpg',
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-38.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-44.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-7.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-12.jpg', 
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-11.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-43.jpg', 
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-21.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-18.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-22.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-116.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-42.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-14.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-39.jpg', '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-54.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-112.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-106.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-25.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-46.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-24.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-29.jpg',
        '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-48.jpg', '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-107.jpg',
        '\\/\\/i0.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-47.jpg', '\\/\\/i2.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-33.jpg',
        '\\/\\/i1.wp.com\\/www.nationalreview.com\\/wp-content\\/uploads\\/2018\\/05\\/harry-meghan-royal-wedding-51.jpg', 'google_ad_cli', 
        'adunit.style.display', '.length', 'google_ad_width', 'google_ad_height', 'adunit.style.margin', 'adunit.style.textalign', 'adunit.classnam',
        '//www.activistpost.com/2018/06/israeli-selling-surveillance-systems-governments-around-world.html', "'theactivistpost", '.innerhtml', 'ld-ajs', 'w.ldadinit=w.ldadinit||', '//www.activistpost.com/2018/06/google-quits-drone-program-u-s-navy-wants-drone-motherships-with-help-of-a-i.html',
        '302px', '422px', '300px', '0px', '12px', '60px', '25px', '250px', '3px', '40px', '1px', '10px','titl', '0\\', '//www.activistpost.com/2018/04/us-bombs-syria-to-cover-up-lack-of-evidence-on-chem-attacks-discredits-own-claims-by-doing-so.html', 
        '//www.activistpost.com/2018/05/red-nose-day-recognizes-kids-living-in-poverty-article-the-rich-get-smart-the-poor-get-technology-the-new-digital-divide-in-school-choice-highlights-part-of-the-problem.html', 
        'firstscript', 'dsqlocal', "'trackback_url", 'disqus_shortnam', 'fb-extra-h2', 'prelimd', 'resize', 'scr+ipt', 'paul', 'watson']
        regex = re.compile(r"\S+www.nationalreview.com\S+")
        linkstr = re.findall(regex, str(werdz))
        say_no.append(linkstr)


        werdz = ([s[-2] for s in werdz if s[-2] not in say_no and 'wp.com' not in s[-2] and 'facebook.com' not in s[-2] and
                 '1897954795849722' not in s[-2] and '999999' not in s[-2] and "'100" not in s[-2] and 'prelimmonth' not in s[-2]
                 and '5g' not in s[-2] and 'scr+ipt' not in s[-2] and 'documentcloud.org' not in s[-2] and 'www.' not in s[-2]
                 and '10 10' not in s[-2] and 'labelmap' not in s[-2] and 'finald' not in s[-2] and '23390304' not in s[-2]
                 and 'box.' not in s[-2] and 'label.' not in s[-2] and '.style' not in s[-2] and '10 100' not in s[-2]
                 and '2c576' not in s[-2] and '2c90' not in s[-2] and 'ryan' not in s[-2] and 'mexico' not in s[-2] and 'todd' not in s[-2]
                 and 'obama' not in s[-2] and 'avenatti' not in s[-2] and 'atblog' not in s[-2] and 'muslim' not in s[-2] and 'dodd' not in s[-2]
                 and 'frank' not in s[-2] and 'rumaihi' not in s[-2] and 'bozorgmehr' not in s[-2] and 'sharafedin' not in s[-2] and 'kardashian' not in s[-2]
                 and 'edit' not in s[-2] and 'splc' not in s[-2] and 'matt' not in s[-2] and 'agorist' not in s[-2] and 'julian' not in s[-2]
                 and 'assange' not in s[-2] and 'wikileaks' not in s[-2] and 'giuliani' not in s[-2] and 'dinesh' not in s[-2] and 'torsion' not in s[-2]
                 and 'freidman' not in s[-2] and 'cia' not in s[-2] and 'tmz' not in s[-2] and 'teacher' not in s[-2] and 'eric' not in s[-2] and 'texas' not in s[-2]
                 and 'god' not in s[-2] and '1864' not in s[-2] and 'abbott' not in s[-2] and 'robert' not in s[-2] and 'derrick' not in s[-2]
                 and 'ajdelgado13' not in s[-2] and 'sach' not in s[-2] and 'cuomo' not in s[-2] and 'orlando' not in s[-2] and 'greiten' not in s[-2] and '10 10' not in s[-2]
                 and 'chizu' not in s[-2] and 'nomiyamaour' not in s[-2] and '000' not in s[-2] and 'yanks' not in s[-2] and 'frompovich' not in s[-2]
                 and 'catherin' not in s[-2] and 'com' not in s[-2] and 'podcast' not in s[-2] and 'kim' not in s[-2] and 'jong' not in s[-2] and 'google_ad_client' not in s[-2]
                 and 'bozorgmehr' not in s[-2] and 'rumaihi' not in s[-2] and 'gowdy' not in s[-2] and 'barr' not in s[-2] and 'roseanne' not in s[-2] and 'mccabe' not in s[-2]
                 and 'andrew' not in s[-2] and 'bitcoin' not in s[-2] and 'nasa' not in s[-2] and 'nixon' not in s[-2] and 'connor' not in s[-2] and 'parenthood' not in s[-2]
                 and 'eagle' not in s[-2] and 'sach' not in s[-2] and 'tusday' not in s[-2] and 'g7' not in s[-2] and 'god' not in s[-2] and '10pm' not in s[-2] and '1am' not in s[-2]
                 and 'yulin' not in s[-2] and 'cruz' not in s[-2] and 'israel' not in s[-2] and '645' not in s[-2] and 'kildani' not in s[-2] and 'house' not in s[-2] and 'vin' not in s[-2]
                 and 'mekelberg' not in s[-2] and '2016' not in s[-2] and '2017' not in s[-2] and '2018' not in s[-2] and '2019' not in s[-2]])
        
        #check cleaned text line from function above
        no_nouns.append(' '.join(werdz))
    return no_nouns

In [ ]:
#Drops nouns and other terms from the text
final_round_clean = drop_nouns(cleaned_text)

In [ ]:
#Fits tfidf vectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 4),  
                                   stop_words='english', 
                                   #token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
cor_tfidf = tfidf.fit_transform(final_round_clean)

## Dimensionality Reduction

In [ ]:
#Fits LSA
lsa = TruncatedSVD(140, algorithm = 'arpack')
corpus_lsa = lsa.fit_transform(cor_tfidf)
corpus_lsa = Normalizer(copy=False).fit_transform(corpus_lsa)

In [ ]:
#Check explained variance
sum(lsa.explained_variance_ratio_)

In [ ]:
#Get terms
terms = tfidf.get_feature_names()

In [ ]:
#check topics modeled
for i, comp in enumerate(lsa.components_):
    Terms_in_Comp = zip(terms,comp)
    sorted_Terms = sorted(Terms_in_Comp, key = lambda x: x[1], reverse=True) [:10]
    print("Topic %d:" %i)
    for term in sorted_Terms:
        print(term[0])
    print(" ")

## Fitting a model

In [ ]:
#Creates test-train split
X = corpus_lsa
y = true_df.source
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#check against dummy classifer
dumb = DummyClassifier(strategy = "stratified", random_state=0)
dumb.fit(X_train, y_train)
print(dumb.score(X_test, y_test))
print(dumb.score(X_train, y_train))

In [ ]:
#Fits a random forest classifier
rf2 = RandomForestClassifier(n_estimators=50)#, max_depth=36)
rf2.fit(X_train, y_train)
print(rf2.score(X_test, y_test))
print(rf2.score(X_train, y_train))

In [ ]:
#Check the confusion matrix
names = ['Fox News', 'National Review', 'Breitbart', 'Info wars', 'Global Research',
       'Activist Post', 'Reuters', 'Associate Press',
       'Alabama Today', 'Huffington Post', 'Daily Beast', 'Mother Jones']
plt.figure(dpi=100)
cm = confusion_matrix(y_test, rf2.predict(X_test), labels =names)
plt.imshow(cm, cmap=plt.cm.Blues)
plt.grid(False)
plt.colorbar();
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], ('FN', 'NR', 'B', 'IW', 'GR',
       'ActP', 'R', 'AP',
       'AT', 'HP', 'DB', 'MJ'))
plt.yticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], ('Fox News', 'National Review', 'Breitbart', 'Info wars', 'Global Research',
       'Activist Post', 'Reuters', 'Associated Press',
       'Alabama Today', 'Huffington Post', 'Daily Beast', 'Mother Jones'));
plt.ylabel("True Source")
plt.xlabel("Predicted Source");
fmt = '.1f'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j]),#, fmt),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

In [ ]:
#Check the classification report
print(classification_report(y_test, rf2.predict(X_test), target_names=names))